In [1]:
%pip install graphene
import pandas as pd
import networkx as nx
from datetime import datetime
from graphene import ObjectType, Schema, String, Float, List, Field, ID
from graphene.types.generic import GenericScalar

# Load and preprocess the deployment data
deployment_df = pd.read_csv('final_dataset.csv')
deployment_df['timestamp'] = pd.to_datetime(deployment_df['timestamp'])

# Load communication data (assuming similar format)
# communication_df = pd.read_csv('communication_data.csv')

class PodMetrics(ObjectType):
    cpu_usage = Float()
    memory_usage = Float()
    network_transmit = Float()

class Pod(ObjectType):
    id = ID()
    name = String()
    metrics = Field(PodMetrics)
    deployed_at = String()
    deployment_time = String()
    status = String()
    microservice = String()
    experiment = String()
    abnormality_class = String()
    communications = List(lambda: CommunicationEdge)
    
    def resolve_communications(self, info):
        # Query communications where this pod is source or target
        pass

class CommunicationEdge(ObjectType):
    source_pod = ID()
    target_pod = ID()
    timestamp = String()
    metrics = GenericScalar()  # Can contain packets, bytes, latency etc.

class Query(ObjectType):
    pod = Field(Pod, id=ID())
    pods_by_time = List(Pod, start=String(), end=String())
    
    def resolve_pod(self, info, id):
        # Get pod data from merged graph
        pass
    
    def resolve_pods_by_time(self, info, start, end):
        # Query pods active in time range
        pass

# Create the merged graph structure
def create_merged_graph(deployment_df, communication_df):
    G = nx.DiGraph()
    
    # Add pods as nodes with deployment info
    for column in deployment_df.columns:
        if column.endswith('_container_cpu_usage_seconds_total'):
            pod_name = column.split('_container_cpu_usage_seconds_total')[0]
            G.add_node(pod_name, 
                      type='pod',
                      cpu_usage=deployment_df[f'{pod_name}_container_cpu_usage_seconds_total'].iloc[0],
                      memory_usage=deployment_df[f'{pod_name}_container_memory_working_set_bytes'].iloc[0],
                      network_transmit=deployment_df[f'{pod_name}_container_network_transmit_packets_total'].iloc[0],
                      deployed_at=deployment_df[f'{pod_name}_deployed_at'].iloc[0],
                      deployment_time=deployment_df['timestamp'].iloc[0].isoformat(),
                      microservice=None,  # Update if microservice info is available
                      experiment=None,    # Update if experiment info is available
                      abnormality_class=None)  # Update if abnormality class info is available
    
    # Add communication edges (example - would need actual comm data)
    # for _, row in communication_df.iterrows():
    #    G.add_edge(row['source'], row['target'], 
    #              timestamp=row['timestamp'],
    #              metrics={'packets': row['packets']})
    
    return G

# Initialize the merged graph
merged_graph = create_merged_graph(deployment_df, None)  # Add communication_df when available

schema = Schema(query=Query)


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


C:\Users\marti\AppData\Local\Temp\ipykernel_33796\3813365407.py:9: DtypeWarning: Columns (29,37,88,92,96,100,128,132,136,140,144,148,152,156,160,164,168,172,176,180,184,188,192,196,200,204,208,212,216,220,224,228,232,236,240,244,248,252,256,260,264) have mixed types. Specify dtype option on import or set low_memory=False.
  deployment_df = pd.read_csv('final_dataset.csv')
